# Example Chapter 5: Using APIs to get Stock Data
In this example we use functions that tap into the yfinance library and Ken French's website to get monthly returns on the Vanguard S\&P 500 ETF (VOO) and short-term US t-bills. We package the yfinance API and the function to scrape Ken French's website in the file "bates_boyer_fletcher_functions.py" for easy use.  Once we have the necessary data in dataframes, we use these data to estimate the annualized expected return, volatility, and Sharpe ratio for VOO. 

In [15]:
# Load in bates_boyer_fletcher_functions.py from the github repository 
!curl -O https://raw.githubusercontent.com/boyerb/Investments/refs/heads/master/functions/bates_boyer_fletcher_functions.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  4128  100  4128    0     0  15085      0 --:--:-- --:--:-- --:--:-- 15345


In [16]:

import bates_boyer_fletcher_functions as invf
import numpy as np
import pandas as pd


In [17]:
# Get monthly returns for a given ticker using the yfinance API
ticker = ['^RUA', 'F']
dat = invf.get_monthly_returns(ticker, '1990-01-01', '2023-12-31', tbill_return=False)
print(dat.head())

[*********************100%***********************]  1 of 1 completed
C:\Users\bhb35\Documents\data\investments\PycharmProjects\Investments\bates_boyer_fletcher_functions.py:86: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  month_end_data = data.resample('M').ffill()  # 'ME' gives month-end, ffill to get last available price
[*********************100%***********************]  1 of 1 completed

             ^RUA         F
Date                       
1990-01 -0.074868  0.005681
1990-02  0.012766  0.066667
1990-03  0.023209  0.027174
1990-04 -0.028393 -0.050828
1990-05  0.087505  0.045326



C:\Users\bhb35\Documents\data\investments\PycharmProjects\Investments\bates_boyer_fletcher_functions.py:86: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  month_end_data = data.resample('M').ffill()  # 'ME' gives month-end, ffill to get last available price


In [18]:
# Calculate the annualized average return and volatility
A_Avg = dat.mean() * 12
A_Vol = dat.std() * np.sqrt(12)
print('Anualized Average Return')
print(A_Avg)
print()
print('Annualized Volatility')
print(A_Vol)

Anualized Average Return
^RUA    0.089472
F       0.133204
dtype: float64

Annualized Volatility
^RUA    0.152483
F       0.431903
dtype: float64


In [19]:
# Download data along with return on short-term tbills from Ken French's website
dat = invf.get_monthly_returns(ticker, '1990-01-01', '2023-12-31', tbill_return=True)
dat['exret0'] = dat[ticker[0]] - dat['RF']
dat['exret1'] = dat[ticker[1]] - dat['RF']
print(dat.head())

[*********************100%***********************]  1 of 1 completed
C:\Users\bhb35\Documents\data\investments\PycharmProjects\Investments\bates_boyer_fletcher_functions.py:86: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  month_end_data = data.resample('M').ffill()  # 'ME' gives month-end, ffill to get last available price
[*********************100%***********************]  1 of 1 completed
C:\Users\bhb35\Documents\data\investments\PycharmProjects\Investments\bates_boyer_fletcher_functions.py:86: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  month_end_data = data.resample('M').ffill()  # 'ME' gives month-end, ffill to get last available price


             ^RUA         F      RF    exret0    exret1
Date                                                   
1990-01 -0.074868  0.005681  0.0057 -0.080568 -0.000019
1990-02  0.012766  0.066667  0.0057  0.007066  0.060967
1990-03  0.023209  0.027174  0.0064  0.016809  0.020774
1990-04 -0.028393 -0.050828  0.0069 -0.035293 -0.057728
1990-05  0.087505  0.045326  0.0068  0.080705  0.038526


In [20]:
# Calcualte and print Sharpe ratios
A_AVG_exret0 = dat['exret0'].mean() * 12
A_Vol_exret0 = dat[ticker[0]].std() * np.sqrt(12)
Sharpe0 = A_AVG_exret0/A_Vol_exret0

A_AVG_exret1 = dat['exret1'].mean() * 12
A_Vol_exret1 = dat[ticker[1]].std() * np.sqrt(12)
Sharpe1 = A_AVG_exret1/A_Vol_exret1

print('Sharpe Ratios')
print(ticker[0], Sharpe0)
print(ticker[1], Sharpe1)



#Sharpe = A_AVG_exret/A_Vol_exret
#print()
#print('Sharpe Ratio: ', Sharpe)

Sharpe Ratios
^RUA 0.418475028174323
F 0.24899623520945782
